In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome(executable_path = r"F:\python\hari\chromedriver.exe")

# URL of the website
url = "www.example.com"

# opening link in the browser
driver.get(url)

# maximizing browser
driver.maximize_window()

# setting driver wait time
wait = WebDriverWait(driver, 15)

C:\Users\prasa\AppData\Local\Temp\ipykernel_7220\2830903261.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = r"F:\python\hari\chromedriver.exe")


In [14]:
house_detial_container = driver.find_element(By.XPATH, '//*[@id="app"]/div/div[3]')

house_detial_container_tag_list = list(house_detial_container.find_elements(By.CLASS_NAME, "swiperHouse"))[1::]

In [19]:
house_detial_link_list = []
for i in house_detial_container_tag_list:
    house_detials_list = i.find_elements(By.CLASS_NAME, "house_image")
    for house_detial in house_detials_list:
        house_detial_link_list.append(house_detial.find_element(By.TAG_NAME, "a").get_attribute('href'))
#print(*house_detial_link_list, sep='\n')

In [91]:
def fetch_detials():
    address_xpath = '//*[@id="app"]/div/div[2]/div[2]/div[1]/div[1]/div[1]/div/div/h1'
    price_xpath = '//*[@id="app"]/div/div[2]/div[2]/div[1]/div[1]/div[1]/div/div/div[1]'

    address_detial_dict = {"address": None, "city_name": None, "house_type": None}
    price_detial_dict = {"price_listing.price_grey": None, "price_sold": None, "list_status_and_list_days": None}

    wait.until(EC.presence_of_element_located((By.XPATH, address_xpath)))
    for key in address_detial_dict:
        try:
            address_tag = driver.find_element(By.XPATH, address_xpath)
            address_detial_dict[key] = address_tag.find_element(By.CLASS_NAME, key).text
        except:
            address_detial_dict[key] = "detial not found"

    wait.until(EC.presence_of_element_located((By.XPATH, price_xpath)))
    for key in price_detial_dict:
        try:
            price_tag = driver.find_element(By.XPATH, price_xpath)
            val = price_tag.find_element(By.CLASS_NAME, key).text
            if key == "list_status_and_list_days":
                if val.startswith("Sold"):
                    price_detial_dict[key] = val.replace("Sold in ", "").replace("Sold ", "")
                else:
                    price_detial_dict[key] = "not sold"
            else:
                price_detial_dict[key] = val.replace("Listed for: ", "").replace("Sold for: ", "")
        except:
            price_tag = driver.find_element(By.XPATH, price_xpath)
            if key == "price_listing.price_grey":
                price_detial_dict[key] = price_tag.find_element(By.CLASS_NAME, "price_listing").text.replace("Listed for: ", "")
            else:
                price_detial_dict[key] = "not sold"

    return address_detial_dict, price_detial_dict

In [ ]:
house_detial_dict = dict()
for idx, link in enumerate(house_detial_link_list):
    driver.get(link)
    detials = fetch_detials()
    house_detial_dict[idx] = {"address": detials[0], "price": detials[1]}

print(house_detial_dict)

In [93]:
import json 

with open("house_detials.json", "w") as outfile:
    json.dump(house_detial_dict, outfile, indent=4)

In [97]:
import csv

def jsonTOcsv(json_array):
    csv = []
    for _, v in json_array.items():
        address = v["address"].values()
        price = v["price"].values()
        csv.append(tuple(address) + ('',) + tuple(price))

    return csv

main_header = ('', 'address', '', '', '', 'price', '')
address_header = ("address", "city_name", "house_type")
price_header = ("price_listing.price_grey", "price_sold", "sold date")
sub_header = address_header + ('',) + price_header

# writing to csv file 
with open("house_detials.csv", 'w') as csvfile: 
    # creating a csv writer object
    csvwrite = csv.writer(csvfile)
        
    # writing the fields 
    csvwrite.writerow(main_header)
    csvwrite.writerow(sub_header)
        
    # writing the data rows 
    csvwrite.writerows(jsonTOcsv(house_detial_dict))